# 🍔 Food Delivery Data Analysis Project

## 📌 Overview
This notebook performs an end-to-end analysis of a food delivery dataset. 
We combine data from three different sources:
1. **`orders.csv`** (Transactions)
2. **`users.json`** (User Demographics)
3. **`restaurants.sql`** (Restaurant Details)

## 🎯 Objectives
- **Data Extraction**: Parse SQL and Load CSV/JSON.
- **Data Transformation**: Merge datasets and handle missing values.
- **Data Analysis**: Answer business questions regarding revenue, customer behavior, and restaurant performance.

In [ ]:
import pandas as pd
import re
import json

# Display settings
pd.set_option('display.max_columns', None)
print("Libraries Imported Successfully")

In [ ]:
# --- Step 1: Load Datasets ---

# 1. Load Orders (CSV)
try:
    orders_df = pd.read_csv('orders.csv')
    print("Orders Data Loaded Successfully. Shape:", orders_df.shape)
except FileNotFoundError:
    print("Error: orders.csv not found.")

# 2. Load Users (JSON)
try:
    users_df = pd.read_json('users.json')
    print("Users Data Loaded Successfully. Shape:", users_df.shape)
except ValueError:
    print("Error: users.json format incorrect.")
except FileNotFoundError:
    print("Error: users.json not found.")

# 3. Load Restaurants (SQL)
# Parsing INSERT statements from the SQL file
restaurants_data = []
try:
    with open('restaurants.sql', 'r') as f:
        content_full = f.read()
        # Regex to capture values inside INSERT INTO restaurants VALUES (...);
        matches = re.findall(r"\((\d+),\s*'([^']*)',\s*'([^']*)',\s*([\d\.]+)\)", content_full)
        
        for m in matches:
            restaurants_data.append({
                'restaurant_id': int(m[0]),
                'restaurant_name': m[1],
                'cuisine': m[2],
                'rating': float(m[3])
            })
    restaurants_df = pd.DataFrame(restaurants_data)
    print("Restaurants Data Parsed Successfully. Shape:", restaurants_df.shape)
except FileNotFoundError:
    print("Error: restaurants.sql not found.")

In [ ]:
# --- Step 2: Merge Datasets ---

# Merge 1: Orders + Users (Left Join on user_id)
merged_df = pd.merge(orders_df, users_df, on='user_id', how='left')

# Merge 2: Result + Restaurants (Left Join on restaurant_id)
merged_df = pd.merge(merged_df, restaurants_df, on='restaurant_id', how='left', suffixes=('', '_sql'))

# --- Step 3: Data Cleaning ---

# Convert order_date to datetime
merged_df['order_date'] = pd.to_datetime(merged_df['order_date'], dayfirst=True)

# Rename 'name' to 'user_name'
merged_df.rename(columns={'name': 'user_name'}, inplace=True)

# Create 'quarter' column
merged_df['quarter'] = merged_df['order_date'].dt.quarter

print("Final Merged Dataset Shape:", merged_df.shape)
print("Columns:", merged_df.columns.tolist())
merged_df.head()

## 📊 Part A: Multiple Choice Questions Analysis
Calculations to determine the correct options for the MCQ section.

In [ ]:
print("--- MCQ Analysis Output ---")

# 1. City with highest revenue from Gold members
q1 = merged_df[merged_df['membership'] == 'Gold'].groupby('city')['total_amount'].sum().sort_values(ascending=False)
print(f"1. Highest Gold Revenue City: {q1.index[0]} (Amount: ₹{q1.iloc[0]:,.2f})")

# 2. Cuisine with highest average order value
q2 = merged_df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)
print(f"2. Highest Avg Order Value Cuisine: {q2.index[0]} (Avg: ₹{q2.iloc[0]:.2f})")

# 3. Users with orders > 1000
user_totals = merged_df.groupby('user_id')['total_amount'].sum()
q3_count = (user_totals > 1000).sum()
print(f"3. Users with total spend > ₹1000: {q3_count}")

# 4. Rating range with highest revenue
bins = [2.9, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0 – 3.5', '3.6 – 4.0', '4.1 – 4.5', '4.6 – 5.0']
merged_df['rating_range'] = pd.cut(merged_df['rating'], bins=bins, labels=labels)
q4 = merged_df.groupby('rating_range', observed=False)['total_amount'].sum().sort_values(ascending=False)
print(f"4. Highest Revenue Rating Range: {q4.index[0]} (Revenue: ₹{q4.iloc[0]:,.2f})")

# 5. Gold members highest avg order value city
q5 = merged_df[merged_df['membership'] == 'Gold'].groupby('city')['total_amount'].mean().sort_values(ascending=False)
print(f"5. Highest Avg Order Value City (Gold): {q5.index[0]} (Avg: ₹{q5.iloc[0]:.2f})")

# 6. Cuisine with lowest distinct restaurants
q6 = merged_df.groupby('cuisine')['restaurant_id'].nunique().sort_values()
print(f"6. Cuisine with Fewest Restaurants: {q6.index[0]} (Count: {q6.iloc[0]})")

# 7. Percentage of Gold orders
gold_pct = (len(merged_df[merged_df['membership'] == 'Gold']) / len(merged_df)) * 100
print(f"7. Percentage of Gold Orders: {gold_pct:.0f}%")

# 8. Restaurant with highest avg value but < 20 orders
rest_stats = merged_df.groupby('restaurant_name').agg({'total_amount': 'mean', 'order_id': 'count'})
q8 = rest_stats[rest_stats['order_id'] < 20].sort_values('total_amount', ascending=False)
print(f"8. Highest Avg Value Restaurant (<20 orders): {q8.index[0]}")

# 9. Top Revenue Combination
q9 = merged_df.groupby(['membership', 'cuisine'])['total_amount'].sum().sort_values(ascending=False)
print(f"9. Highest Revenue Combination: {q9.index[0]}")

# 10. Quarter with highest revenue
q10 = merged_df.groupby('quarter')['total_amount'].sum().sort_values(ascending=False)
print(f"10. Highest Revenue Quarter: Q{q10.index[0]}")

## 🔢 Part B: Numerical Questions
Precise calculations for the numerical answer section.

In [ ]:
print("--- Numerical Answers Output ---")

# 1. Total Gold Orders
ans1 = len(merged_df[merged_df['membership'] == 'Gold'])
print(f"1. Total Gold Orders: {ans1}")

# 2. Hyderabad Revenue
ans2 = merged_df[merged_df['city'] == 'Hyderabad']['total_amount'].sum()
print(f"2. Hyderabad Revenue (rounded): {round(ans2)}")

# 3. Distinct Users
ans3 = merged_df['user_id'].nunique()
print(f"3. Distinct Users: {ans3}")

# 4. Gold Avg Order Value
ans4 = merged_df[merged_df['membership'] == 'Gold']['total_amount'].mean()
print(f"4. Gold Avg Order Value (2 decimals): {round(ans4, 2)}")

# 5. Orders Rating >= 4.5
ans5 = len(merged_df[merged_df['rating'] >= 4.5])
print(f"5. Orders with Rating >= 4.5: {ans5}")

# 6. Orders in Top Gold City
gold_df = merged_df[merged_df['membership'] == 'Gold']
top_city = gold_df.groupby('city')['total_amount'].sum().idxmax()
ans6 = len(gold_df[gold_df['city'] == top_city])
print(f"6. Orders in Top Gold Revenue City ({top_city}): {ans6}")

## 📝 Part C: Fill-in-the-Blank Verification
Checking data properties to answer the fill-in-the-blank questions.

In [ ]:
print("--- Verification Output ---")
print(f"Join Column used: user_id")
print(f"Format of restaurant file: SQL")
print(f"Total Rows in Dataset: {len(merged_df)}")
print(f"Missing values handling: NaN")
print(f"Pandas function for joining: merge")
print(f"Membership column source: users.json")
print(f"Restaurant join key: restaurant_id")
print(f"Food type column: cuisine")
print("Multiple orders result in repeated user details: True")

## 💾 Export Final Dataset
Saving the processed dataframe to a CSV file for submission.

In [ ]:
merged_df.to_csv('final_food_delivery_dataset.csv', index=False)
print("Success: 'final_food_delivery_dataset.csv' created.")